In [1]:
import asyncio # 비동기화 모듈
from bleak import BleakScanner # BLE 검색 모듈
import nest_asyncio
nest_asyncio.apply()

In [3]:
# 비동기 형태로 BLE 장치 검색

async def runna():
    # 검색 시작 (검색이 종료될때까지 대기)
    # 기본 검색 시간은 5초이다.
    devices = await BleakScanner.discover()
    # 검색된 장치들 리스트 출력
    for d in devices:
        print(d)

# 비동기 이벤트 루프 생성
loop = asyncio.get_event_loop()
# 비동기 형태로 run(검색)함수 실행
# 완료될때까지 대기
loop.run_until_complete(runna())

69:E7:26:83:E5:4B: Apple, Inc. (b'\x10\x06\x1c\x19\xc4S\xe5\x08')
73:60:A5:3F:12:31: Apple, Inc. (b'\x10\x07\t\x1b\x88=q\x0c\x18')
DB:15:C2:8A:93:DF: Mi Band 3


In [8]:
import asyncio
from bleak import BleakClient

address = "DB:15:C2:8A:93:DF"
async def run(address):    
    async with BleakClient(address) as client:
        print('connected')
        services = await client.get_services()        
        for service in services:
            print(service)             
            # 서비스의 UUID 출력   
            print('\tuuid:', service.uuid)
            print('\tcharacteristic list:')
            # 서비스의 모든 캐릭터리스틱 출력용
            for characteristic in service.characteristics:
                # 캐릭터리스틱 클래스 변수 전체 출력
                print('\t\t', characteristic)
                # UUID 
                print('\t\tuuid:', characteristic.uuid)
                # decription(캐릭터리스틱 설명)
                print('\t\tdescription :', characteristic.description)
                # 캐릭터리스틱의 속성 출력
                # 속성 값 : ['write-without-response', 'write', 'read', 'notify']
                print('\t\tproperties :', characteristic.properties)

    print('disconnect')

loop = asyncio.get_event_loop()
loop.run_until_complete(run(address))
print('done')

connected
00001800-0000-1000-8000-00805f9b34fb (Handle: 1): Generic Access Profile
	uuid: 00001800-0000-1000-8000-00805f9b34fb
	characteristic list:
		 00002a00-0000-1000-8000-00805f9b34fb (Handle: 2): Device Name
		uuid: 00002a00-0000-1000-8000-00805f9b34fb
		description : Device Name
		properties : ['read']
		 00002a01-0000-1000-8000-00805f9b34fb (Handle: 4): Appearance
		uuid: 00002a01-0000-1000-8000-00805f9b34fb
		description : Appearance
		properties : ['read']
		 00002a04-0000-1000-8000-00805f9b34fb (Handle: 6): Peripheral Preferred Connection Parameters
		uuid: 00002a04-0000-1000-8000-00805f9b34fb
		description : Peripheral Preferred Connection Parameters
		properties : ['read']
00001801-0000-1000-8000-00805f9b34fb (Handle: 8): Generic Attribute Profile
	uuid: 00001801-0000-1000-8000-00805f9b34fb
	characteristic list:
		 00002a05-0000-1000-8000-00805f9b34fb (Handle: 9): Service Changed
		uuid: 00002a05-0000-1000-8000-00805f9b34fb
		description : Service Changed
		properties : ['

In [11]:
import asyncio
from bleak import BleakClient

address = "DB:15:C2:8A:93:DF"
# 읽기/쓰기용 캐릭터리스틱 uuid
notity_charcteristic_uuid = "00002a37-0000-1000-8000-00805f9b34fb"

def notify_callback(sender: int, data: bytearray):
    print('sender: ', sender, 'data: ', data)

async def run(address):              
    # BleakClient 클래스 생성 및 바로 연결 시작
    # address: ESP32 맥주소
    # timeout: 연결 제한 시간 5초가 넘어가면 더 이상 연결하지 말고 종료
    async with BleakClient(address, timeout=60.0) as client:                    
        # 연결을 성공함
        print('connected')
        # 연결된 BLE 장치의 서비스 요청
        services = await client.get_services()
        # 서비스들을 루프돌려 내부 캐릭터리스틱 정보 조회
        for service in services:
            print('service uuid:', service.uuid)
            # 각 서비스들에 있는 캐릭터리스틱을 루프 돌려 속성들 파악하기
            for characteristic in service.characteristics:
                print('  uuid:', characteristic.uuid)
                # handle 정보도 함께 확인
                print('  handle:', characteristic.handle) 
                print('  properties: ', characteristic.properties)
                # 캐릭터리스틱 UUID가 우리가 찾는 UUID인지 먼저 확인
                if characteristic.uuid == notity_charcteristic_uuid:  
                    # 우리가 찾던 UUID가 맞다면 
                    # 해당 캐릭터리스틱에 notify 속성이 있는지 확인
                    if 'notify' in characteristic.properties:
                        # notify 속성이 있다면 BLE 장치의 notify 속성을 
                        # 활성화 작업 후 notify_callback 함수 연결                        
                        print('try to activate notify.')
                        await client.start_notify(characteristic.uuid, notify_callback)

        # client 가 연결된 상태라면        
        if client.is_connected:
            # 1초간 대기
            await asyncio.sleep(1) 
            print('try to deactivate notify.')
            # 활성시켰단 notify를 중지 시킨다.
            await client.stop_notify(notity_charcteristic_uuid)

    #with 문을 빠져나오면 장치는 알아서 disconnect가 된다.
    print('disconnect')


loop = asyncio.get_event_loop()
loop.run_until_complete(run(address))
print('done')

connected
service uuid: 00001800-0000-1000-8000-00805f9b34fb
  uuid: 00002a00-0000-1000-8000-00805f9b34fb
  handle: 2
  properties:  ['read']
  uuid: 00002a01-0000-1000-8000-00805f9b34fb
  handle: 4
  properties:  ['read']
  uuid: 00002a04-0000-1000-8000-00805f9b34fb
  handle: 6
  properties:  ['read']
service uuid: 00001801-0000-1000-8000-00805f9b34fb
  uuid: 00002a05-0000-1000-8000-00805f9b34fb
  handle: 9
  properties:  ['read', 'indicate']
service uuid: 0000180a-0000-1000-8000-00805f9b34fb
  uuid: 00002a25-0000-1000-8000-00805f9b34fb
  handle: 13
  properties:  ['read']
  uuid: 00002a27-0000-1000-8000-00805f9b34fb
  handle: 15
  properties:  ['read']
  uuid: 00002a28-0000-1000-8000-00805f9b34fb
  handle: 17
  properties:  ['read']
  uuid: 00002a23-0000-1000-8000-00805f9b34fb
  handle: 19
  properties:  ['read']
  uuid: 00002a50-0000-1000-8000-00805f9b34fb
  handle: 21
  properties:  ['read']
service uuid: 00001530-0000-3512-2118-0009af100700
  uuid: 00001531-0000-3512-2118-0009af10

BleakDotNetTaskError: System.Exception: 특성을 쓸 수 없습니다. (예외가 발생한 HRESULT: 0x80650003)